## Step 5 Run BikewaySim

1. Process network spatial data into a routable network graph format
2. Reconcile networks into one through node and link overlap conflation
3. Create final network graph and calculate link costs
4. Create OD tables
5. __Run BikewaySim__

## Import Modules

In [ ]:
import geopandas as gpd
from pathlib import Path

In [ ]:
from bikewaysim_lite import *

## Import Files

In [ ]:
#define fp names
import_fp = Path.home() / Path('Documents/BikewaySimData')
export_fp = Path.home() / Path('Documents/BikewaySimData/transport_geos')

ods = pd.read_csv(r'bikewaysim_outputs/samples_in/all_tazs.csv')
links = gpd.read_file(r'trb2023/network.gpkg',layer='links')
improved_links = gpd.read_file('trb2023/network.gpkg',layer='imp_links')
#should also have a seperate layer of improved nodes
#imp_nodes = gpd.read_file('trb2023/network.gpkg',layer='imp_nodes')
nodes = gpd.read_file(r'trb2023/network.gpkg',layer='nodes')
tazs = gpd.read_file(r'trb2023/tazs.gpkg',layer='tazs')

#reduce size (for testing)
#ods=ods.iloc[0:100,:]

## Snapping

In [ ]:
#get network nodes
ods = snap_ods_to_network(ods,nodes)
#need one for improved nodes
#ods_improved = snap_ods_to_network(ods,improved_nodes)

In [ ]:
#shortest distance version
ods_dist, links = find_shortest(links, nodes, ods, 'dist')

In [ ]:

#impedance version
ods_imp, links = find_shortest(links, nodes, ods, 'imp')

In [ ]:

#network improvements (replace with imp_nodes)
ods_improved, improved_links = find_shortest(improved_links, nodes, ods, 'imp')

## Betweenness Centrality

In [ ]:
#find difference in btw centrality
links['btw_difference'] = links['dist_btw_cntrlty'] - links['per_dist_btw_cntrlty']

## Percent Detour

In [ ]:
#cacluate pct detour
ods_imp, by_taz_imp = percent_detour(ods_dist,ods_imp,tazs)
ods_improved, by_taz_improved = percent_detour(ods_dist,ods_imp,tazs)

## Basic Stats

In [ ]:
#basic stats
#find avg impedance
ods.dist.mean()
ods.per_dist.mean()
ods.imp_dist.mean()

# Bikesheds

In [ ]:
bikeshed = {}
taz = '553'
radius = 10
buffer_size = 100

#distance impedance
bikeshed, bikeshed_node = make_bikeshed(links,nodes,taz,ods,radius,buffer_size,'dist')

#attribute impedance
bikeshed, bikeshed_node = make_bikeshed(links,nodes,taz,ods,radius,buffer_size,'dist')

#network improvement
bikeshed, bikeshed_node = make_bikeshed(imp_links,nodes,taz,ods,radius,buffer_size,'dist')

bikeshed_node.to_file(output_fp/Path('bikesheds.gpkg'),layer='node',driver='GPKG')

### Find Changes in Bikesheds

In [ ]:

#find removed links
rem_links = bikeshed['dist'][-(bikeshed['dist']['A_B'].isin(bikeshed['per_dist']['A_B']))]
rem_links.to_file(output_fp/Path('bikesheds.gpkg'),layer='removed')

#find added links
add_links = bikeshed['per_dist'][-bikeshed['per_dist']['A_B'].isin(bikeshed['dist']['A_B'])]
add_links.to_file(output_fp/Path('bikesheds.gpkg'),layer='add')

#find added links from improvement
add_impr = bikeshed['imp_dist'][-bikeshed['imp_dist']['A_B'].isin(bikeshed['per_dist']['A_B'])]
add_impr.to_file(output_fp/Path('bikesheds.gpkg'),layer='imp add')